In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import urljoin
import pandas as pd

In [3]:
url_base = 'https://www.chicagomag.com/'
url_sub = 'Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
url = url_base + url_sub

In [5]:
html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

In [6]:
rest_lists = soup.select('.sammy')
rest_lists[0]

<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br>
Old Oak Tap<br>
<em>Read more</em> </br></br></a></div>
</div>

In [7]:
print(soup.find_all('div', 'sammy'))

[<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br>
Old Oak Tap<br>
<em>Read more</em> </br></br></a></div>
</div>, <div class="sammy" style="position: relative;">
<div class="sammyRank">2</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/"><b>Fried Bologna</b><br/>
Au Cheval<br/>
<em>Read more</em> </a></div>
</div>, <div class="sammy" style="position: relative;">
<div class="sammyRank">3</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/"><b>Woodland Mushroom</b><br/>
Xoco<br/>
<em>Read more</em> </a></div>
</div>, <div class="sammy" style="position: relative;">
<div class="sammyRank">4</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Al

In [8]:
len(soup.find_all('div', 'sammy'))

50

In [9]:
print(soup.find_all('div', 'sammy')[0])

<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br>
Old Oak Tap<br>
<em>Read more</em> </br></br></a></div>
</div>


In [10]:
tmp_one = soup.find_all('div', 'sammy')[0]
type(tmp_one)
# bs4.element.Tag는 그 변수에 다시 태그로 찾는 (find, find_all) 명령을 사용할 수 있다.

bs4.element.Tag

In [11]:
tmp_one.find(class_='sammyRank')

<div class="sammyRank">1</div>

In [12]:
tmp_one.find(class_='sammyRank').get_text()

'1'

In [13]:
tmp_one.find(class_='sammyListing').get_text()

'BLT\r\nOld Oak Tap\nRead more '

In [14]:
tmp_one.find('a')['href']

'/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [15]:
import re

In [16]:
tmp_string = tmp_one.find(class_='sammyListing').get_text()

re.split(('\n|\r\n'), tmp_string)

print(re.split(('\n|\r\n'), tmp_string)[0])
print(re.split(('\n|\r\n'), tmp_string)[1])

BLT
Old Oak Tap


In [17]:
cafe_name = []; rank = []; url_add = []; main_menu = []

list_soup = soup.find_all('div', 'sammy')
for item in list_soup:
    
    tmp_string = item.find(class_='sammyListing').get_text()

    rank.append(int(item.find(class_='sammyRank').get_text()))
    main_menu.append(re.split(('\n|\r\n'), tmp_string)[0])
    cafe_name.append(re.split(('\n|\r\n'), tmp_string)[1])

    url_add.append(urljoin(url_base, item.find('a')['href']))
    # urljoin은 Full-path와 partial-path를 하나로 통일해준다.

In [18]:
import pandas as pd

cafe = pd.DataFrame({
    'Rank': rank,
    'Cafe': cafe_name,
    'Menu': main_menu,
    'URL': url_add
})
cafe.head()

,Rank,Cafe,Menu,URL
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...


In [19]:
cafe.to_csv('chicago.csv', sep=',', encoding='utf-8')

In [20]:
len(rank), len(cafe_name), len(main_menu), len(url_add)

(50, 50, 50, 50)

### 다수의 웹 페이지에 자동으로 접근해서 원하는 정보 가져오기

In [21]:
html = urlopen(cafe['URL'][0])
soup_tmp = BeautifulSoup(html, 'html.parser')

In [22]:
soup_tmp.select_one('.addy')

<p class="addy">
<em>$10. 2109 W. Chicago Ave., 773-772-0406, <a href="http://www.theoldoaktap.com/">theoldoaktap.com</a></em></p>

In [23]:
tmp = soup_tmp.select_one('.addy').get_text().split(',')
len(tmp)
# .sting()은 중간에 태그가 있는경우 나오지 않는다. 주의하기!!
# 3은 홈페이지가 있고, 2는 홈페이지가 없다.

3

In [24]:
tmp[0]

'\n$10. 2109 W. Chicago Ave.'

In [25]:
price = tmp[0].split()[0][:-1]
price

'$10'

In [26]:
tmp[1].strip()

'773-772-0406'

In [27]:
addr = ' '.join(tmp[0].split()[1:])
addr

'2109 W. Chicago Ave.'

In [28]:
tmp[0], tmp[-1], tmp[-2][:-1]

('\n$10. 2109 W. Chicago Ave.', ' theoldoaktap.com', ' 773-772-040')

In [29]:
price = tmp[0].split()[0][:-1]
price

'$10'

In [30]:
from tqdm import tqdm_notebook

In [34]:
rest_list = soup.select('.sammy')
rest_list[0]

<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br>
Old Oak Tap<br>
<em>Read more</em> </br></br></a></div>
</div>

In [35]:
rank_list = []; url_list = []; menu_list = []; cafe_list = []
price_list = []; addr_list = []; tel_list = []; hp_list = []
for rest in tqdm_notebook(rest_list):
    rank = int(rest.select_one('.sammyRank').string)
    url = urljoin(url_base, rest.find('a')['href'])
    tmp = rest.find('a').get_text().split('\n')
    menu = tmp[0].replace('\r', '')
    cafe = tmp[1]
    rank_list.append(rank)
    url_list.append(url)
    menu_list.append(menu)
    cafe_list.append(cafe)

    html = urlopen(url)
    soup_tmp = BeautifulSoup(html, 'html.parser')
    tmp = soup_tmp.select_one('.addy').get_text().split(',')
    price = tmp[0].split()[0][:-1]
    price_list.append(price)
    addr = ' '.join(tmp[0].split()[1:])
    if len(tmp) == 1:
        addr_list.append(addr)
        tel_list.append(' ')
        hp_list.append(' ')
    elif len(tmp) == 2:
        if addr.find('Multiple') >= 0:
            addr_list.append(' ')
            tel_list.append(' ')
            hp_list.append(tmp[1])
        else:
            addr_list.append(addr)
            tel_list.append(tmp[1].strip())
            hp_list.append(' ')
    elif len(tmp) == 3:
        addr_list.append(addr)
        tel_list.append(tmp[1].strip())
        hp_list.append(tmp[2])
    elif len(tmp) == 4:
        addr += ', ' + tmp[1]
        addr_list.append(addr)
        tel_list.append(tmp[2].strip())
        hp_list.append(tmp[3])

In [36]:
df2 = pd.DataFrame({
    'Rank': rank_list,
    'Cafe': cafe_list,
    'Menu': menu_list,
    'Price': price_list,
    'Address': addr_list,
    'Telephone': tel_list,
    'Home Page': hp_list
})
df2.head()

,Rank,Cafe,Menu,Price,Address,Telephone,Home Page
0,1,Old Oak Tap,BLT,$10,2109 W. Chicago Ave.,773-772-0406,theoldoaktap.com
1,2,Au Cheval,Fried Bologna,$9,800 W. Randolph St.,312-929-4580,aucheval.tumblr.com
2,3,Xoco,Woodland Mushroom,$9.50,445 N. Clark St.,312-334-3688,rickbayless.com
3,4,Al’s Deli,Roast Beef,$9.40,"914 Noyes St., Evanston",847-475-9400,alsdeli.net
4,5,Publican Quality Meats,PB&L,$10,825 W. Fulton Mkt.,312-445-8977,publicanqualitymeats.com


In [92]:
df2.to_csv('chicago2.csv', sep=',', encoding='utf-8')